# model data processing

env : retfound

In [3]:
import numpy as np
import pandas as pd

from skimage.io import imread 
from skimage.transform import resize

In [4]:
csv_read = pd.read_csv("/home/john/network/RETFound/data/RegressionData.csv")

# features _ Train

In [5]:
def extract_features(image_path):
    image = imread(image_path)
    image = resize(image, (224, 224)) 

    # 一维向量
    feature_vector = image.flatten()
    
    return feature_vector


file_paths = csv_read['Image_Path']
age_label = csv_read['RealAge'] 

# Image resize & flatten
features = np.array([extract_features(image_path) for image_path in file_paths])

In [4]:
file_paths[0]

'/home/john/network/RETFound/data/RegressionTrainingData/4385815-02-L-20170802.jpg'

In [5]:
len(features)

5214

# features _ validation

In [6]:
vali_csv_read = pd.read_csv("/home/john/network/RETFound/data/RegressionData_vali.csv")

In [9]:
def extract_features_vali(image_path):
    image = imread(image_path)
    image = resize(image, (224, 224)) 

    feature_vector = image.flatten()
    
    return feature_vector


file_paths_vali = vali_csv_read['Image_Path']
age_label_vali = vali_csv_read['RealAge'] 

features_vali = np.array([extract_features_vali(image_path) for image_path in file_paths_vali])

In [50]:
len(features_vali)

1289

In [8]:
file_paths_vali[0]

'/home/john/network/RETFound/data/RegressionvalidationData/2310298-05-L-20220708.jpg'

In [10]:
len(features_vali)

1289

In [ ]:
# import torch.utils.data as data
# from  torchvision import datasets,transforms

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(features, age_label, test_size=0.2, random_state=42)

In [ ]:
# print(len(X_train))
# print(len(X_test))

In [21]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# SVR

In [ ]:
# model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

# model.fit(X_train, y_train)

# predictions = model.predict(X_test)

# mse = mean_squared_error(y_test, predictions)
# print("MSE:", mse)

# XGBoost

In [11]:
X_train = features
X_test = features_vali
y_train = age_label
y_test = age_label_vali

In [12]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

5214 1289 5214 1289


In [13]:
from xgboost.sklearn import XGBRegressor

In [14]:
model = XGBRegressor(
                                                max_depth = 8,
                                                n_estimators = 500, #400
                                                subsample = 0.7,
                                                colsample_bytree = 0.7,
                                                learning_rate = 0.005, #0.05
                                                n_jobs = -1,
                                                random_state = 42
                                                )

In [15]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.005, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [16]:
import joblib

In [17]:
joblib.dump(model, 'xgb_model.pkl')

['xgb_model.pkl']

In [51]:
loaded_model = joblib.load('xgb_model.pkl')

In [19]:
predictions = loaded_model.predict(X_test)

In [22]:
print("MAE:", mean_absolute_error(y_test,predictions))
print("MSE:", mean_squared_error(y_test,predictions))
print("RMSE:", np.sqrt(mean_squared_error(y_test,predictions)))

MAE: 9.329031942055334
MSE: 139.7859244481505
RMSE: 11.82310976216285


In [23]:
for i in range(0,6):
    print(predictions[i])

63.872295
60.26348
53.811493
61.070713
58.879055
65.03735


In [24]:
for i in range(0,6):
    print(y_test[i])

59
68
82
66
56
77


# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

param_grid = {
    'learning_rate': [0.03, 0.05, 0.1],
    'max_depth': [1,3,5,7,9],
    'n_estimators': [100, 200, 300]
}

xgb_model = XGBRegressor()

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

In [ ]:
print('Best parameter : [{}]\n'.format(grid_search.best_params_))